<a href="https://colab.research.google.com/github/HerraKaava/tensorflow/blob/main/tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3 align="left">Workflow</h3>

In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:

* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
print(tf.__version__)

2.15.0


<br>

<h1 align"center"=>Introduction to Tensors</h1>

<h3>Creating tensors with tf.constant()</h3>

In [5]:
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [6]:
# Check the number of dimensions of a tensor

scalar.ndim

0

In [7]:
# Create a vector

vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [8]:
# Check the dimension of our vector

vector.ndim

1

In [9]:
# Create a matrix

matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [10]:
matrix.ndim

2

In [11]:
# Let's create another matrix with flaot16 dtype

matrix2 = tf.constant([[10., 7.],
                       [3., 2.],
                       [8., 9.]],
                      dtype=tf.float16)
matrix2

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

* By default, dtype with tf.constant() is int32 (32-bit precision).
* The higher the number of precision, the more exact these numbers are stored on your computer.
* So float16 < float32 for the same number n.
* This also means that the lower the precision, the less space it takes to store it on your computer.

In [12]:
matrix2.ndim

2

In [13]:
# Let's create a tensor (ndim > 2)

tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                     [[7, 8, 9],
                      [10, 11, 12]]])
tensor

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]], dtype=int32)>

In [14]:
tensor.ndim

3

* The number of elements in the shape object correspond to the number of dimensions in a tensor.
* Notice that the dimensions of a tensor are affected by where you place the square brackets.

What we've created so far:

* scalar: a single number
* vector: a number with direction (e.g., wind speed and direction)
* matrix: a 2-dimensional array of numbers
* tensor: an n-dimensional array of numebrs


Notice that

* 0-dimensional tensor is a scalar
* 1-dimensional tensor is a vector
* 2-dimensional tensor is a matrix

<br>

<h3>Creating tensors with tf.Variable()</h3>

In [15]:
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])

In [16]:
# Let's try to change one of the elements in our changeable tensor

changeable_tensor[0].assign(7)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

* Notice that *tf.variable()* objects are **mutable**.
* In Python, an object is considered mutable, if its value can be changed after creating it.
* Notice that the assign() function assigns values in-place.

In [17]:
# Uncomment and run to see the error message

# unchangeable_tensor[0].assign(7)

* AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'
* This is because *tf.constant()* objects are **immutable**.
* In Python, an object is considered immutable, if its value cannot be changed after it has been created.

**NOTE**

* Rarely in practice you'll need to decide whether to use *tf.constant()* or *tf.variable()* to create tensors, as TensorFlow does this for you.

<br>

<h3>Creating random tensors</h3>

* Random tensors are tensors of some arbitrary size with random numbers.

In [18]:
# Initialize a new random number generator using the seed value 42.
# This part does not initialize any tensor values.
# Here we only set up a generator that can be used to create random values

rng = tf.random.Generator.from_seed(42)

In [19]:
# Assign random data from a standard normal distribution into the rng object

rng = rng.normal(shape=(3,2))

In [20]:
rng

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [21]:
# Shuffle the order of elements in a tensor

arr = tf.constant([[10,7],
                   [3,4],
                   [2,5]])

tf.random.shuffle(arr)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

* Note that random.shuffle() shuffles the elements along its first dimension.
* Since the first dimension stands for rows (3), only the rows gets shuffled, and the order of the elements in the rows stays the same.

If you want to get the same results for reproducibility, you'll have to set a global random seed.

In [22]:
# Global random seed

tf.random.set_seed(42)
tf.random.shuffle(arr)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [ 2,  5],
       [10,  7]], dtype=int32)>

* This produces the same pseudo random numbers every time the code is ran.

In [23]:
tf.random.shuffle(arr, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

* This does not produce the same pseudo random numbers, because specifying the seed number inside a function is an operation level seed (not global).
* If both the global and the operation seed are set, both seeds are used in conjunction to determine the random sequence.
* This means that specifying both global and operation level seeds will produce different pseudo random numbers than just specifying a global random seed.
* [tf.random.set_seed](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)

<br>

<h3>Creating tensors from NumPy arrays</h3>

In [24]:
# Create a tensor of ones

tf.ones([4, 4])

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [25]:
# Create a tensor of zeros

tf.zeros(shape=(4,4))

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

**Turning NumPy arrays into tensors**

* The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing).

In [26]:
numpy_arr = np.arange(1, 25, dtype=np.int32)

In [27]:
tf.constant(numpy_arr)

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

* As seen above, NumPy arrays can be directly converted into tf tensors.
* We can also modify the shape of the converted numpy array as long as the size matches the new dimensions.
* For example, since *numpy_arr* has size 24, we can change its shape into (2,3,4), because $2*3*4 = 24.$

In [29]:
tf.constant(numpy_arr, shape=(2,3,4))

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

In [31]:
tf.constant(numpy_arr, shape=(3,8), dtype=np.int32)

<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16],
       [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>

<br>

<h3>Getting information from your tensors</h3>

Some important tensor attributes include

* shape
  * the length (number of elements) of each of the dimensions of a tensor.

* rank
  * the number of tensor dimensions.

* axis or dimension
  * a particular dimension of a tensor.

* size
  * the total number of elements in the tensor.

In [32]:
# Create a rank 4 tensor

t4 = tf.zeros(shape=(2,3,4,5))
t4

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [33]:
t4.ndim

4

In [37]:
# Play with this indexing, and you'll understand how to slice tensors.

t4[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [40]:
tf.size(t4)

<tf.Tensor: shape=(), dtype=int32, numpy=120>

In [49]:
# To obtain just the size itself, use the numpy() attribute.

tf.size(t4).numpy()

120

In [42]:
2*3*4*5

120

In [47]:
t4[0, 0, :, :]

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

<br>

<h3>Indexing tensors</h3>

In [52]:
t4.shape

TensorShape([2, 3, 4, 5])

In [50]:
# Get the first 2 elements of each dimension of the t4 (rank 4) tensor

t4[:2, :2, :2, :2]


<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [57]:
# Get the first element from each dimension from each index except the last one

t4[:1, :1, :1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

* Notice that if we do not specify what to get from the last dimension, all of it gets returned.

In [60]:
# This returns the same tensor as the one above

t4[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [64]:
# Create a rank 2 tensor (2 dimensions)

t2 = tf.constant([[10, 7],
                  [3, 4]])
print(t2)
print(t2.ndim)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)
2


In [67]:
# Get the last item of each row of our rank 2 tensor (i.e., get the last column)

t2[:, 1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [68]:
# Add a new dimension to our rank 2 tensor

t2[..., tf.newaxis]

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [81]:
# This will also produce a new axis, but in a different place

t2[:, tf.newaxis]

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]], dtype=int32)>

* Notice that you need the three dots (...) in order to get a new dimension as the last dimension.

In [82]:
# Alternative to [..., tf.newaxis]

tf.expand_dims(t2, axis=-1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

* axis=-1 stands for the last axis (i.e., the new axis should be placed as the last axis).

In [84]:
# We could also place the new axis as the first axis

tf.expand_dims(t2, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

* The only difference between these two last lines of code (axis=-1 vs. axis=0) is that the numbers are stored differently.

<br>

<h3>Manipulating tensors (basic tensor operations)</h3>

**Basic operations** (element-wise operations)

*  +, -, *, /

In [86]:
# You can add values to a tensor

t = tf.constant([[10, 7],
                 [5, 6]])
t+5

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[15, 12],
       [10, 11]], dtype=int32)>

* Addition to a tensor adds the value to every element of the tensor.

In [87]:
# Multiplication

t * 2

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 14],
       [10, 12]], dtype=int32)>

In [88]:
# Subtraction

t - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-5, -4]], dtype=int32)>

In [89]:
# Division

t / 2

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[5. , 3.5],
       [2.5, 3. ]])>

In [90]:
# We can use the tensorflow built-in function too

tf.multiply(t, 2)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 14],
       [10, 12]], dtype=int32)>

* If you have to use some sort of operations on your tensor, always choose a function that comes with the tensorflow library (if possible), since those operations run on the GPU and therefore are faster.

<br>

<h3>Matrix multiplication</h3>

In machine learning, matrix multiplication is one of the most common tensor operations. When multiplying two matrices A and B, the matrix B needs to have the same number of rows than the matrix A has columns. That is, if A has the dimensions $m \times n$ and B has the dimensions $k \times p,$ the matrix multiplication AB is defined if and only if $n = k.$ Note that in matrix multiplication the order matters, meaning that $AB \neq BA$ (well sometimes it is, but in most cases, it is not). The resulting matrix AB will have dimensions $m \times p$.

In [91]:
# Matrix multiplication in tensorflow

tf.linalg.matmul(t2, t2)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [92]:
t2 @ t2

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [94]:
np.matmul(t2, t2)

array([[121,  98],
       [ 42,  37]], dtype=int32)

In [101]:
X1 = tf.constant([[4, 5, 2],
                  [9, 4, 1],
                  [5, 6, 7]])

X2 = tf.constant([[7, 8, 9],
                  [5, 5, 3]])

print(f'{X1.shape}\n{X2.shape}')

(3, 3)
(2, 3)


* These matrices cannot be multiplied, because X1 has 3 columns and X2 has only 2 rows ($2 \neq 3$).
* If you try to multiply these as they are, you will get an error.

In [102]:
# X2 needs to have 3 rows for the matrix multiplication, so let's reshape it

tf.matmul(X1, tf.reshape(X2, shape=(3,2)))

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 83,  63],
       [104,  95],
       [124,  91]], dtype=int32)>

* Notice that the new dimensions of the matrix X1X2 are $3 \times 2$.
* This shows that the resulting matrix after a matrix multiplication AB will have the same amount of rows as A, and same amount of columns as B.

In [103]:
# Transpose

tf.transpose(X1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[4, 9, 5],
       [5, 4, 6],
       [2, 1, 7]], dtype=int32)>

In [112]:
tf.transpose(X2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[7, 5],
       [8, 5],
       [9, 3]], dtype=int32)>

<br>

<h3>The dot product</h3>

Matrix multiplication is also referred to as the *dot product*.

You can perform matlix multiplication using:

* tf.matmul()
* tf.tensordot()


In [104]:
X1

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[4, 5, 2],
       [9, 4, 1],
       [5, 6, 7]], dtype=int32)>

In [107]:
tf.matmul(X1, tf.reshape(X2, shape=(3,2)))

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 83,  63],
       [104,  95],
       [124,  91]], dtype=int32)>

In [111]:
tf.tensordot(X1, tf.reshape(X2, shape=(3,2)), axes=1)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 83,  63],
       [104,  95],
       [124,  91]], dtype=int32)>

* Notice that with tf.tensordot(), you need to specify along which axis to perform the multiplication.

<br>

<h3>Changing the datatype of tensors</h3>

In [126]:
print(tf.constant([7.4, 5.0]))
print(tf.constant([5, 4]))
print(tf.constant([1.0, 2, 3, 4, 5]))

tf.Tensor([7.4 5. ], shape=(2,), dtype=float32)
tf.Tensor([5 4], shape=(2,), dtype=int32)
tf.Tensor([1. 2. 3. 4. 5.], shape=(5,), dtype=float32)


* As seen above, when including floats into the tensor, the default datatype will be float32.
* On the other hand, if the tensor contains only integers, the default datatype will be int32.
* Note that if the tensor contains even one float, all of the numbers will be converted into floats.

In [130]:
# Let's reduce the precision from float32 --> float16

print(tf.constant([1.0, 2.0], dtype=tf.float16))

tf.Tensor([1. 2.], shape=(2,), dtype=float16)


* [Mixed precision](https://www.tensorflow.org/guide/mixed_precision)
* The link above contains important information on the different datatypes.

<br>

<h3>Aggregating tensors</h3>